In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


xtrain = pd.read_csv("/kaggle/input/titanic/train.csv",index_col = 0)
xtest = pd.read_csv("/kaggle/input/titanic/test.csv",index_col = 0)

y=xtrain.Survived
xtrain.drop(['Survived'], axis=1, inplace=True)

x_trainfull, x_val, y_train, y_val = train_test_split(xtrain, y, test_size=0.2, random_state=0)

In [3]:
num_cols = x_trainfull.select_dtypes(exclude=['object']).columns
cat_cols = x_trainfull.select_dtypes(include=['object']).columns

fullcols = num_cols.append(cat_cols)

x_train = x_trainfull[fullcols].copy()
x_val = x_val[fullcols].copy()
x_test = xtest[fullcols].copy()



In [4]:
print(num_cols)
print(cat_cols)
print(fullcols)
print(f'\nx_train shape: {xtrain.shape}\n')
print(f'x_train columns: {xtrain.columns}')

Index(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')
Index(['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'], dtype='object')
Index(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Name', 'Sex', 'Ticket',
       'Cabin', 'Embarked'],
      dtype='object')

x_train shape: (891, 10)

x_train columns: Index(['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare',
       'Cabin', 'Embarked'],
      dtype='object')


In [5]:
x_train.head()

,Pclass,Age,SibSp,Parch,Fare,Name,Sex,Ticket,Cabin,Embarked
PassengerId,,,,,,,,,,
141,3,NaN,0,2,15.2458,"Boulos, Mrs. Joseph (Sultana)",female,2678,NaN,C
440,2,31.0,0,0,10.5000,"Kvillner, Mr. Johan Henrik Johannesson",male,C.A. 18723,NaN,S
818,2,31.0,1,1,37.0042,"Mallet, Mr. Albert",male,S.C./PARIS 2079,NaN,C
379,3,20.0,0,0,4.0125,"Betros, Mr. Tannous",male,2648,NaN,C
492,3,21.0,0,0,7.2500,"Windelov, Mr. Einar",male,SOTON/OQ 3101317,NaN,S


In [6]:
print(num_cols)
print(cat_cols)

Index(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')
Index(['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'], dtype='object')


In [7]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

numericalTransformer = SimpleImputer(strategy='median')


categoricalTransformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numericalTransformer, num_cols),
        ('cat', categoricalTransformer, cat_cols)
    ])

myPipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', XGBClassifier(n_estimators=500, learning_rate=0.05,
                                  max_depth=4,random_state=42,eval_metric='logloss'))])


myPipeline.fit(x_train,y_train)

preds = myPipeline.predict(x_val)

print('Accuracy:', accuracy_score(y_val, preds))



Accuracy: 0.8547486033519553


In [8]:
prediction = myPipeline.predict(x_test)

In [9]:
# Save test predictions to file
output = pd.DataFrame({'PassengerId': x_test.index,
                       'Survived': prediction})
output.to_csv('submission.csv', index=False)